### Dump 2024 icy TOTDs to DB

In [1]:
import csv
from datetime import datetime
from typing import List
from sqlalchemy.exc import IntegrityError
from db import Map, get_session

totd_maps: List[Map] = []

with open('./dat/ice_totds.csv', mode='r') as f:
    csv_reader = csv.DictReader(f)
    for row in csv_reader:
        totd_maps.append(
            Map(
                uid = row['MAP_UID'],
                name = row['MAP_NAME'],
                totd_date = datetime.strptime(row['TOTD_DATE'], '%Y-%m-%d'),
            )
        )

with get_session() as session:
    session.add_all(totd_maps)
    try:
        session.commit()
        print(f"Added {len(totd_maps)} maps")
    except IntegrityError as ex:
        print(f"Unique constraint failed - maps already loaded")
        session.rollback()

Added 16 maps


### Dump PIL S2 maps to DB

In [10]:
import csv
from datetime import datetime
from typing import List
from sqlalchemy.exc import IntegrityError
from db import Map, get_session

totd_maps: List[Map] = []

with open('./dat/pil_maps.csv', mode='r') as f:
    csv_reader = csv.DictReader(f)
    for row in csv_reader:
        totd_maps.append(
            Map(
                uid = row['MAP_UID'],
                name = row['MAP_NAME'],
            )
        )

with get_session() as session:
    session.add_all(totd_maps)
    try:
        session.commit()
        print(f"Added {len(totd_maps)} maps")
    except IntegrityError as ex:
        print(f"Unique constraint failed - maps already loaded")
        session.rollback()

Added 3 maps


### Resolve player account_id values from player names

In [2]:
import csv
from typing import List

csv_rows:List[dict] = []

with open('./dat/players.csv', mode='r') as f:
    csv_reader = csv.DictReader(f)
    for row in csv_reader:
        csv_rows.append(row)

print(f"Found {len(csv_rows)} players")

Found 170 players


In [3]:
import nadeo

AUTH_TOKEN = nadeo.public_authenticate()

In [4]:
import requests

fail_count = 0

for row in csv_rows:
    player_name = row['PLAYER_NAME']
    headers = {'Authorization': AUTH_TOKEN}
    url = f"https://api.trackmania.com/api/display-names/account-ids?displayName[]={player_name}"
    resp = requests.get(url=url, headers=headers)
    resp.raise_for_status()
    j = resp.json()
    try:
        row['ACCOUNT_ID'] = j[player_name]
    except TypeError:
        fail_count += 1
        print(f"Failed to resolve account_id for {player_name}")

print(f"Failed to resolve {fail_count} of {len(csv_rows)} player account_id values")

Failed to resolve account_id for bead_Sillypants
Failed to resolve 1 of 170 player account_id values


In [6]:
fieldnames = ['TEAM_NAME','PLAYER_NAME','ACCOUNT_ID']
    
with open('./dat/players_cleaned.csv', mode='w', newline='\n') as f:
    writer = csv.DictWriter(f, fieldnames=fieldnames)
    writer.writeheader()  # Write the header row
    
    for row in csv_rows:
        data = {**row}
        del data['PLAYER_UID']
        writer.writerow(data)

print(f"Wrote {len(csv_rows)} rows")

Wrote 170 rows


### Dump fully curated players/teams list to DB

In [7]:
import csv
from sqlalchemy.exc import IntegrityError
from db import Player, Team, get_session

num_teams = 0
num_players = 0
last_team_name = None
with get_session() as session:
    with open('./dat/players_curated.csv', 'r') as f:
        csv_reader = csv.DictReader(f)
        for row in csv_reader:    
            team_name = row['TEAM_NAME']
            if team_name != last_team_name:
                team = Team(name = team_name)
                last_team_name = team_name
                session.add(team)
                session.flush()
                session.refresh(team)
                num_teams += 1
            player = Player(
                account_id = row['ACCOUNT_ID'],
                username = row['PLAYER_NAME'],
                team_id = team.id,
            )
            num_players += 1
            session.add(player)
    try:
        session.commit()
        print(f"Added {num_players} players from {num_teams} teams")
    except IntegrityError as ex:
        print(f"Unique constraint failed - players already loaded")
        session.rollback()

Added 170 players from 29 teams


### Gather records from all maps and dump to DB

In [11]:
import nadeo

AUTH_TOKEN = nadeo.live_authenticate()

In [12]:
import requests
from db import Map, Player, Record, get_session

with get_session() as session:
    maps = [m._asdict() for m in session.query(Map.uid, Map.name).filter(Map.totd_date == None).order_by(Map.totd_date).all()]
    for m in maps:
        map_uid = m['uid']
        map_name = m['name']
        num_records_added = 0
        for offset in range(10):
            length = 100
            print(f"Retrieving records {length*offset+1} - {length*(offset+1)} for map \"{map_name}\"")
            url = f"https://live-services.trackmania.nadeo.live/api/token/leaderboard/group/Personal_Best/map/{map_uid}/top?onlyWorld=true&length={length}"
            headers = {'Authorization': AUTH_TOKEN}
            resp = requests.get(url=url, headers=headers)
            resp.raise_for_status()
            j = resp.json()
            entries = j['tops'][0]['top']
            for entry in entries:
                record = Record(
                    account_id = entry['accountId'],
                    map_uid = map_uid,
                    time = entry['score'] / 1000.0,
                    position = entry['position'],
                )
                session.add(record)
                num_records_added += 1
        session.commit()
        print(f"Added {num_records_added} records for map \"{map_name}\"")
        print('---')

Retrieving records 1 - 100 for map "Nelja"
Retrieving records 101 - 200 for map "Nelja"
Retrieving records 201 - 300 for map "Nelja"
Retrieving records 301 - 400 for map "Nelja"
Retrieving records 401 - 500 for map "Nelja"
Retrieving records 501 - 600 for map "Nelja"
Retrieving records 601 - 700 for map "Nelja"
Retrieving records 701 - 800 for map "Nelja"
Retrieving records 801 - 900 for map "Nelja"
Retrieving records 901 - 1000 for map "Nelja"
Added 1000 records for map "Nelja"
---
Retrieving records 1 - 100 for map "Legacy"
Retrieving records 101 - 200 for map "Legacy"
Retrieving records 201 - 300 for map "Legacy"
Retrieving records 301 - 400 for map "Legacy"
Retrieving records 401 - 500 for map "Legacy"
Retrieving records 501 - 600 for map "Legacy"
Retrieving records 601 - 700 for map "Legacy"
Retrieving records 701 - 800 for map "Legacy"
Retrieving records 801 - 900 for map "Legacy"
Retrieving records 901 - 1000 for map "Legacy"
Added 1000 records for map "Legacy"
---
Retrieving re